In [38]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from utils.util import file2list
import numpy as np
import math
from copy import deepcopy

In [39]:
class node:
    def __init__(self, p = None, l = None, r = None):
        self.p = p
        self.l = l
        self.r = r
class vnode:
    def __init__(self, p, v):
        self.p = p
        self.v = v
    

def str_node(root):
    if type(root) == vnode: return str(root.v)
    return "[{},{}]".format(str_node(root.l), str_node(root.r))

def parse(n,p):
    if len(n) == 1:
        return vnode(p, int(n))
    level = 0
    for i,c in enumerate(n):
        if c == '[':
            level += 1
        elif c == ']':
            level -= 1
        elif c == ',' and level == 1:
            current = node(p)
            current.l = parse(n[1:i], current)
            current.r = parse(n[i+1:-1], current)
            return current

def add_right(s, v):
    c = s
    p = c.p
    while p.p and p.r == c:
        c = p
        p = c.p
    if p.p == None and p.r == c: return
    c = p.r
    while type(c) != vnode:
        c = c.l
    c.v += v
        

def add_left(s, v):
    c = s
    p = c.p
    while p.p and p.l == c:
        c = p
        p = c.p
    if p.p == None and p.l == c: return
    c = p.l
    while type(c) != vnode:
        c = c.r
    c.v += v

exploded_flag = False
def explode(n, level=0):
    if type(n) == vnode: return
    global exploded_flag
    if (level >= 4 and type(n) == node and
        type(n.l) == vnode and type(n.r) == vnode):
        p1 = n.l.v
        p2 = n.r.v
        add_right(n, p2)
        add_left(n, p1)
        new_n = vnode(n.p, 0)
        if n.p.l == n: n.p.l = new_n
        else: n.p.r = new_n
        exploded_flag = True
        return
    if not exploded_flag: explode(n.l, level+1)
    if not exploded_flag: explode(n.r, level+1)

split_flag = False
def split(n):
    global split_flag
    if type(n) == vnode:
        if n.v >= 10:
            p = n.p
            p1 = math.floor(n.v/2)
            p2 = math.ceil(n.v/2)
            new_n = node(p)
            new_n.l = vnode(new_n, p1)
            new_n.r = vnode(new_n, p2)
            if n.p.l == n: n.p.l = new_n
            else: n.p.r = new_n
            split_flag = True
        return
    if not split_flag: split(n.l)
    if not split_flag: split(n.r)

def norm(n):
    global exploded_flag
    global split_flag
    while True:
        exploded_flag = False
        split_flag = False
        explode(n)
        if exploded_flag:
            continue
        split(n)
        if not exploded_flag and not split_flag: break
    return n

def add(n1, n2):
    n = node(None, n1, n2)
    n1.p = n
    n2.p = n
    return norm(n)

def magn(n):
    if type(n) == vnode: return n.v
    return 3*magn(n.l) + 2*magn(n.r)


In [46]:
numbers = file2list("./input.txt")
res = parse(numbers[0], None)
for i in range(1, len(numbers)):
    n = parse(numbers[i], None)
    res = add(res, n)
print(str_node(res))
print(magn(res))

[[[[7,7],[8,8]],[[8,9],[7,8]]],[[[6,7],[0,8]],[8,8]]]
3793


In [49]:
numbers = file2list("./input.txt")
sum = 0
for i in range(0, len(numbers)):
    for j in range(0, len(numbers)):
        if i == j: continue
        i_n = parse(numbers[i], None)
        j_n = parse(numbers[j], None)
        sum_n = magn(add(i_n, j_n))
        if sum_n > sum: sum = sum_n
print(sum)

4695
